## 1. Import thư viện, chuẩn bị dữ liệu

In [1]:
import os
import sys
import time
import argparse
import shutil
from os import makedirs
from os.path import exists
import re
import string
import struct
from copy import deepcopy
import copy
import random
from random import shuffle
import numpy as np 
import pandas as pd 
import numpy as np
from numpy.random import random as rand
import math
import pickle
import torch
import torch as T
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import operator

In [2]:
class CommonConfigs(object):
    def __init__(self, d_type):
        self.ROOT_PATH = os.getcwd() + "/"  # Lấy đường dẫn thư mục hiện tại
        self.TRAINING_DATA_PATH = self.ROOT_PATH + d_type + "/train_set/"  # Đường dẫn tập huấn luyện
        self.VALIDATE_DATA_PATH = self.ROOT_PATH + d_type + "/validate_set/"  # Đường dẫn tập validation
        self.TESTING_DATA_PATH = self.ROOT_PATH + d_type + "/test_set/"  # Đường dẫn tập kiểm tra
        self.RESULT_PATH = self.ROOT_PATH + d_type + "/result/"  # Đường dẫn thư mục lưu kết quả
        self.MODEL_PATH = self.ROOT_PATH + d_type + "/model/"  # Đường dẫn thư mục lưu mô hình
        self.BEAM_SUMM_PATH = self.RESULT_PATH + "/beam_summary/"  # Đường dẫn lưu kết quả beam search
        self.BEAM_GT_PATH = self.RESULT_PATH + "/beam_ground_truth/"  # Đường dẫn lưu ground truth beam search
        self.GROUND_TRUTH_PATH = self.RESULT_PATH + "/ground_truth/"  # Đường dẫn ground truth
        self.SUMM_PATH = self.RESULT_PATH + "/summary/"  # Đường dẫn lưu tóm tắt
        self.TMP_PATH = self.ROOT_PATH + d_type + "/tmp/"  # Đường dẫn thư mục tạm

class DeepmindTraining(object):
    IS_UNICODE = False  # Setup sử dụng Unicode không
    REMOVES_PUNCTION = False  # Loại bỏ dấu câu hay không
    HAS_Y = True  # Có nhãn Y hay không
    BATCH_SIZE = 32  # Kích thước batch

class DeepmindTesting(object):
    IS_UNICODE = False  # Setup sử dụng Unicode không
    HAS_Y = True  # Có nhãn Y hay không
    BATCH_SIZE = 100  # Kích thước batch
    MIN_LEN_PREDICT = 35  # Độ dài tối thiểu của đầu ra dự đoán
    MAX_LEN_PREDICT = 120  # Độ dài tối đa của đầu ra dự đoán
    MAX_BYTE_PREDICT = None  # Giới hạn số byte đầu ra (None là không giới hạn)
    PRINT_SIZE = 500  # Số lượng mẫu hiển thị khi in kết quả
    REMOVES_PUNCTION = False  # Loại bỏ dấu câu hay không

class DeepmindConfigs():
    cc = CommonConfigs("so")  # Cấu hình chung cho tập dữ liệu "so"

    CELL = "gru"  # Loại cell RNN sử dụng (GRU hoặc LSTM)
    CUDA = True  # Sử dụng GPU hay không
    COPY = True  # Sử dụng cơ chế Copy hay không
    COVERAGE = True  # Sử dụng cơ chế Coverage hay không
    BI_RNN = True  # Sử dụng RNN hai chiều hay không
    BEAM_SEARCH = True  # Sử dụng Beam Search hay không
    BEAM_SIZE = 4  # Kích thước Beam Search
    AVG_NLL = False  # Sử dụng trung bình Negative Log-Likelihood hay không
    NORM_CLIP = 2  # Giới hạn giá trị norm của gradient
    if not AVG_NLL:
        NORM_CLIP = 5  # Nếu không dùng trung bình NLL thì đặt giá trị norm_clip là 5
    LR = 0.15  # Learning rate

    DIM_X = 100  # Số chiều embedding đầu vào
    DIM_Y = DIM_X  # Số chiều embedding đầu ra

    MIN_LEN_X = 16  # Độ dài tối thiểu của đầu vào
    MIN_LEN_Y = 4  # Độ dài tối thiểu của đầu ra
    MAX_LEN_X = 128  # Độ dài tối đa của đầu vào
    MAX_LEN_Y = 64  # Độ dài tối đa của đầu ra
    MIN_NUM_X = 1  # Số lượng tối thiểu của đầu vào
    MAX_NUM_X = 1  # Số lượng tối đa của đầu vào
    MAX_NUM_Y = None  # Số lượng tối đa của đầu ra (None là không giới hạn)

    NUM_Y = 1  # Số lượng đầu ra
    HIDDEN_SIZE = 256  # Kích thước hidden state của RNN

    UNI_LOW_FREQ_THRESHOLD = 10  # Ngưỡng tần suất thấp cho từ vựng

    PG_DICT_SIZE = 60000  # Kích thước từ điển của Pointer-Generator

    W_UNK = "<unk>"  # Token unknown
    W_BOS = "<bos>"  # Token bắt đầu câu
    W_EOS = "<eos>"  # Token kết thúc câu
    W_PAD = "<pad>"  # Token padding
    W_LS = "<s>"  # Token bắt đầu câu (có thể dùng riêng)
    W_RS = "</s>"  # Token kết thúc câu (có thể dùng riêng)

class SOConfigs():
    cc = CommonConfigs("so")  # Cấu hình chung cho tập dữ liệu "so"

    CELL = "gru"  # Loại cell RNN sử dụng (GRU hoặc LSTM)
    CUDA = True  # Sử dụng GPU hay không
    COPY = True  # Sử dụng cơ chế Copy hay không
    COVERAGE = True  # Sử dụng cơ chế Coverage hay không
    BI_RNN = True  # Sử dụng RNN hai chiều hay không
    BEAM_SEARCH = True  # Sử dụng Beam Search hay không
    BEAM_SIZE = 5  # Kích thước Beam Search
    AVG_NLL = False  # Sử dụng trung bình Negative Log-Likelihood hay không
    NORM_CLIP = 2  # Giới hạn giá trị norm của gradient
    if not AVG_NLL:
        NORM_CLIP = 5  # Nếu không dùng trung bình NLL thì đặt giá trị norm_clip là 5
    LR = 0.15  # Learning rate

    DIM_X = 100  # Số chiều embedding đầu vào
    DIM_Y = DIM_X  # Số chiều embedding đầu ra

    MIN_LEN_X = 16  # Độ dài tối thiểu của đầu vào
    MIN_LEN_Y = 4  # Độ dài tối thiểu của đầu ra
    MAX_LEN_X = 128  # Độ dài tối đa của đầu vào
    MAX_LEN_Y = 64  # Độ dài tối đa của đầu ra
    MIN_NUM_X = 1  # Số lượng tối thiểu của đầu vào
    MAX_NUM_X = 1  # Số lượng tối đa của đầu vào
    MAX_NUM_Y = None  # Số lượng tối đa của đầu ra (None là không giới hạn)

    NUM_Y = 1  # Số lượng đầu ra
    HIDDEN_SIZE = 256  # Kích thước hidden state của RNN

    UNI_LOW_FREQ_THRESHOLD = 10  # Ngưỡng tần suất thấp cho từ vựng

    PG_DICT_SIZE = 50000  # Kích thước từ điển của Pointer-Generator

    W_UNK = "<unk>"  # Token unknown
    W_BOS = "<bos>"  # Token bắt đầu câu
    W_EOS = "<eos>"  # Token kết thúc câu
    W_PAD = "<pad>"  # Token padding
    W_LS = "<s>"  # Token bắt đầu câu (có thể dùng riêng)
    W_RS = "</s>"  # Token kết thúc câu (có thể dùng riêng)


In [3]:
stop_words = {"-lrb-", "-rrb-", "-"}
unk_words = {"unk", "<unk>"}

def load_vocab(src_path, tgt_path):
    """
    Đọc từ vựng từ file nguồn và file đích, tạo một từ điển đếm số lần xuất hiện của từng từ.
    """
    vocab_dict = {}
    vocab_list = []
    with open(src_path, 'r') as src:
        for line in src:
            words = line.lower().split()
            for word in words:
                if word not in vocab_dict:
                    vocab_dict[word] = 1
                    vocab_list.append(word)
                else:
                    vocab_dict[word] += 1

    with open(tgt_path, 'r') as src:
        for line in src:
            words = line.lower().split()
            for word in words:
                if word not in vocab_dict:
                    vocab_dict[word] = 1
                    vocab_list.append(word)
                else:
                    vocab_dict[word] += 1
    return vocab_dict, vocab_list
    pass

def to_dict(xys, dic):
    # Tập từ điển không bao gồm tập kiếm tra (dict should not consider test set)!!!!!
    """
    Cập nhật từ điển từ vựng với dữ liệu từ tập huấn luyện.
    """
    for xy in xys:
        sents, summs = xy
        y = summs[0]
        for w in y:
            if w in dic:
                dic[w] += 1
            else:
                dic[w] = 1

        x = sents[0]
        for w in x:
            if w in dic:
                dic[w] += 1
            else:
                dic[w] = 1
    return dic

def prepare_dict(src_path, tgt_path, train_xy_list, configs):
    """
    Xây dựng từ điển từ vựng với giới hạn số lượng từ nhất định.
    """
    print ("fitering and building dict...")
    use_abisee = True
    all_dic1 = {}
    all_dic2 = {}
    dic_list = []
    all_dic1, dic_list = load_vocab(src_path, tgt_path)
    all_dic2 = to_dict(train_xy_list, all_dic2)
    for w, tf in all_dic2.items():
        if w not in all_dic1:
            all_dic1[w] = tf

    candiate_list = dic_list[0:configs.PG_DICT_SIZE] # 50000
    candiate_set = set(candiate_list)

    dic = {}
    w2i = {}
    i2w = {}
    w2w = {}

    for w in [configs.W_PAD, configs.W_UNK, configs.W_EOS]:
        w2i[w] = len(dic)
        i2w[w2i[w]] = w
        dic[w] = 10000
        w2w[w] = w

    for w, tf in all_dic1.items():
        if w in candiate_set:
            w2i[w] = len(dic)
            i2w[w2i[w]] = w
            dic[w] = tf
            w2w[w] = w
        else:
            w2w[w] = configs.W_UNK

    hfw = []
    sorted_x = sorted(dic.items(), key=operator.itemgetter(1), reverse=True)
    for w in sorted_x:
        hfw.append(w[0])

    assert len(hfw) == len(dic)
    assert len(w2i) == len(dic)
    return all_dic1, dic, hfw, w2i, i2w, w2w


def get_xy_tuple(cont, head, configs):
    """
    Chuyển đổi nội dung từ câu thành tuple (đoạn văn bản gốc, câu tóm tắt).
    """
    x = read_cont(cont, configs)
    y = read_head(head, configs)

    if x != None and y != None:
        return (x, y)
    else:
        return None

def read_cont(src_line, cfg):
    """
    Xử lý văn bản nguồn (đoạn văn gốc) và thêm token kết thúc câu.
    """
    lines = []
    line = src_line #del_num(f_cont)
    words = line.split()
    lines += words
    lines += [cfg.W_EOS]
    return (lines, src_line)

def read_head(tgt_line, cfg):
    """
    Xử lý tiêu đề hoặc câu tóm tắt với token kết thúc câu.
    """
    lines = []
    line = tgt_line
    words = line.split()
    lines += words
    lines += [cfg.W_EOS]
    return (lines, tgt_line)

def load_lines(src_path, tgt_path,  configs):
    """
    Đọc dữ liệu từ file nguồn và file đích, sau đó chuyển đổi thành danh sách tuple (input, summary).
    """
    lines = []
    with open(src_path, 'r') as src_file, open(tgt_path, 'r') as tgt_file:
        for src_line, tgt_line in zip(src_file, tgt_file):

            src_line = src_line.strip().lower()
            tgt_line = tgt_line.strip().lower()
            xy_tuple = get_xy_tuple(src_line, tgt_line, configs)
            lines.append(xy_tuple)
    return lines


def prepare_dir():
   return configs

"""
Chuẩn bị và xử lý dữ liệu tập huấn luyện, kiểm thử, và validation.
"""
# Prepare Dirs
configs = DeepmindConfigs()
TRAINING_PATH = configs.cc.TRAINING_DATA_PATH
VALIDATE_PATH = configs.cc.VALIDATE_DATA_PATH
TESTING_PATH = configs.cc.TESTING_DATA_PATH
RESULT_PATH = configs.cc.RESULT_PATH
MODEL_PATH = configs.cc.MODEL_PATH
BEAM_SUMM_PATH = configs.cc.BEAM_SUMM_PATH
BEAM_GT_PATH = configs.cc.BEAM_GT_PATH
GROUND_TRUTH_PATH = configs.cc.GROUND_TRUTH_PATH
SUMM_PATH = configs.cc.SUMM_PATH
TMP_PATH = configs.cc.TMP_PATH

print ("train: " + TRAINING_PATH)
print ("test: " + TESTING_PATH)
print ("validate: " + VALIDATE_PATH)
print ("result: " + RESULT_PATH)
print ("model: " + MODEL_PATH)
print ("tmp: " + TMP_PATH)

if not exists(TRAINING_PATH):
    makedirs(TRAINING_PATH)
if not exists(VALIDATE_PATH):
    makedirs(VALIDATE_PATH)
if not exists(TESTING_PATH):
    makedirs(TESTING_PATH)
if not exists(RESULT_PATH):
    makedirs(RESULT_PATH)
if not exists(MODEL_PATH):
    makedirs(MODEL_PATH)
if not exists(BEAM_SUMM_PATH):
    makedirs(BEAM_SUMM_PATH)
if not exists(BEAM_GT_PATH):
    makedirs(BEAM_GT_PATH)
if not exists(GROUND_TRUTH_PATH):
    makedirs(GROUND_TRUTH_PATH)
if not exists(SUMM_PATH):
    makedirs(SUMM_PATH)
if not exists(TMP_PATH):
    makedirs(TMP_PATH)

data_dir = r"C:\Users\Acer\Desktop\2024\lab\Code2que\Challenge Task 2 - Code2Que\data\archive\Code2Que-data\javadata"
# Prepare Dataset
src_path = f"{data_dir}/src-train.txt"
tgt_path = f"{data_dir}/tgt-train.txt"

#train
train_xy_list = load_lines(src_path, tgt_path,  configs)

train_xy_list, val_xy_list = train_test_split(train_xy_list, test_size=0.2, random_state=42)
print(f"Train set size: {len(train_xy_list)}")
print(f"Validation set size: {len(val_xy_list)}")
#dictionary
all_dic1, dic, hfw, w2i, i2w, w2w = prepare_dict(src_path, tgt_path, train_xy_list, configs)

#test
src_path = f"{data_dir}/src-test-clear.txt"
tgt_path = f"{data_dir}/tgt-test-clear.txt"
test_xy_list = load_lines(src_path, tgt_path,  configs)


train: c:\Users\Acer\Desktop\2024\lab\Code2que/so/train_set/
test: c:\Users\Acer\Desktop\2024\lab\Code2que/so/test_set/
validate: c:\Users\Acer\Desktop\2024\lab\Code2que/so/validate_set/
result: c:\Users\Acer\Desktop\2024\lab\Code2que/so/result/
model: c:\Users\Acer\Desktop\2024\lab\Code2que/so/model/
tmp: c:\Users\Acer\Desktop\2024\lab\Code2que/so/tmp/
Train set size: 200566
Validation set size: 50142
fitering and building dict...


In [4]:
# Lấy một cặp từ train_xy_list
sample_pair = train_xy_list[0]

# In ra cấu trúc của cặp dữ liệu
print("Cấu trúc của một cặp (x, y):")
print(f"x (source code): {sample_pair[0][0][:20]}... (đã cắt ngắn)")
print(f"y (question): {sample_pair[1][0][:20]}... (đã cắt ngắn)")

# In ra văn bản gốc đầy đủ
print("\nĐoạn code đầy đủ:")
print(sample_pair[0][1])

# In ra câu hỏi đầy đủ
print("\nCâu hỏi đầy đủ:")
print(sample_pair[1][1])

Cấu trúc của một cặp (x, y):
x (source code): ['long', 'starttime', '=', 'system', '.', 'currenttimemillis', '();', 'while', '(', 'running', ')', '{', 'track', '.', 'repaint', '();', '//', 'varying', 'the', 'x']... (đã cắt ngắn)
y (question): ['how', 'to', 'round', 'seconds', 'to', 'tenths', 'place', '?', '<eos>']... (đã cắt ngắn)

Đoạn code đầy đủ:
long starttime = system . currenttimemillis (); while ( running ) { track . repaint (); // varying the x position movement horse . setx ( xpos += ( int ) ( math . random () * num + num )); // sleeping the thread try { thread . sleep ( 100 ); } catch ( interruptedexception e ) { e . printstacktrace (); } if ( xpos >= finish_line ) { running = false ; long endtime = system . currenttimemillis (); joptionpane . showmessagedialog ( new jframe (), id + str +( float )( starttime - endtime )/ 1000 *- 1 . 0 + str ); } }

Câu hỏi đầy đủ:
how to round seconds to tenths place ?


## 2. Chuẩn bị mô hình CodeBERT(encoder) - LSTM + Attention (decoder)
- Encoder: Biến code thành token ID (1, sentence_len) -> Embedding (1, sentence_len, 768) -> CodeBERT layers -> Output là last hidden state (1, sentenece_len, 768)
- Decoder: Nhúng token đầu vào thành vector -> Tính attention score với h_t và encoder output -> Context vector -> Ghép với cả embedding đầu vào cho LSTM (Từ trước) -> Tính xác suất sinh từ trên từ điển (lấy từ tập các câu hỏi) dùng greedy search


In [5]:
# CodeBERT LSTM Attention Model
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaModel, RobertaTokenizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class CodeBERTEncoder(nn.Module):
    """
    Encoder sử dụng mô hình CodeBERT từ Microsoft Research
    """
    def __init__(self, freeze_encoder=False):
        super(CodeBERTEncoder, self).__init__()
        self.codebert = RobertaModel.from_pretrained("microsoft/codebert-base")

        # Cố định các tham số của CodeBERT nếu cần
        if freeze_encoder:
            for param in self.codebert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        """
        :param input_ids: Token IDs của đoạn code [batch_size, seq_length]
        :param attention_mask: Attention mask [batch_size, seq_length]
        :return: Tất cả hidden states từ CodeBERT
        """
        outputs = self.codebert(input_ids=input_ids, attention_mask=attention_mask)
        # Lấy tất cả các hidden states
        hidden_states = outputs.last_hidden_state  # [batch_size, seq_length, hidden_size]
        return hidden_states

class BahdanauAttention(nn.Module):
    """
    Cơ chế Attention dựa trên Bahdanau et al. (2015)
    """
    def __init__(self, encoder_dim, decoder_dim):
        super(BahdanauAttention, self).__init__()
        self.encoder_dim = encoder_dim  # Kích thước hidden của encoder (768)
        self.decoder_dim = decoder_dim  # Kích thước hidden của decoder (256)

        # Mạng fully connected để tính attention scores
        self.attn = nn.Linear(self.encoder_dim + self.decoder_dim, self.decoder_dim)
        self.v = nn.Linear(self.decoder_dim, 1, bias=False)

    def forward(self, encoder_outputs, decoder_hidden, mask=None):
        """
        :param encoder_outputs: Outputs của encoder [batch_size, src_seq_len, encoder_dim]
        :param decoder_hidden: Hidden state của decoder [batch_size, decoder_dim]
        :param mask: Mask cho padding tokens [batch_size, src_seq_len]
        """
        src_len = encoder_outputs.shape[1]
        batch_size = encoder_outputs.shape[0]

        # Mở rộng decoder_hidden để kết hợp với từng token đầu ra của encoder
        decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)  # [batch_size, src_len, decoder_dim]

        # Concatenate encoder outputs và decoder hidden state
        attn_inputs = torch.cat((decoder_hidden, encoder_outputs), dim=2)  # [batch_size, src_len, encoder_dim + decoder_dim]

        # Tính attention scores
        energy = torch.tanh(self.attn(attn_inputs))  # [batch_size, src_len, decoder_dim]
        attention = self.v(energy).squeeze(2)  # [batch_size, src_len]

        # Áp dụng mask nếu có
        if mask is not None:
            attention = attention.masked_fill(mask == 0, -1e10)

        # Chuyển thành phân phối xác suất qua softmax
        attention_weights = F.softmax(attention, dim=1)  # [batch_size, src_len]

        # Tính weighted sum
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs)  # [batch_size, 1, encoder_dim]
        context_vector = context_vector.squeeze(1)  # [batch_size, encoder_dim]

        return context_vector, attention_weights

class LSTMDecoder(nn.Module):
    """
    Decoder sử dụng LSTM với cơ chế attention
    """
    def __init__(self,
                 vocab_size,
                 embed_dim,
                 encoder_dim=768,
                 hidden_dim=256,
                 num_layers=2,
                 dropout=0.5):
        super(LSTMDecoder, self).__init__()

        self.vocab_size = vocab_size  # Kích thước từ điển đầu ra
        self.embed_dim = embed_dim    # Dimension của word embeddings
        self.encoder_dim = encoder_dim  # Dimension của encoder outputs (768 cho CodeBERT)
        self.hidden_dim = hidden_dim  # Dimension của decoder hidden state
        self.num_layers = num_layers  # Số lớp LSTM

        # Embedding layer cho từ đầu ra
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Cơ chế attention
        self.attention = BahdanauAttention(encoder_dim, hidden_dim)

        # Projection layer để giảm kích thước của encoder outputs
        self.encoder_projection = nn.Linear(encoder_dim, hidden_dim)

        # LSTM decoder
        self.lstm = nn.LSTM(
            input_size=embed_dim + encoder_dim,  # Đầu vào là concatenation của embedding và context vector
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )

        # Lớp output projection
        self.output_projection = nn.Linear(hidden_dim + encoder_dim, vocab_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def init_hidden(self, encoder_outputs):
        """
        Khởi tạo hidden state của LSTM từ encoder outputs
        """
        # Sử dụng CLS token (vị trí 0) để khởi tạo
        cls_token = encoder_outputs[:, 0, :]  # [batch_size, encoder_dim]

        # Project xuống kích thước hidden của decoder
        h0 = self.encoder_projection(cls_token)
        h0 = h0.unsqueeze(0).repeat(self.num_layers, 1, 1)  # [num_layers, batch_size, hidden_dim]
        c0 = torch.zeros_like(h0)

        return (h0, c0)

    def forward_step(self, input_token, last_hidden, encoder_outputs, src_mask=None):
        """
        Một bước decoder cho một token đầu vào
        """
        # Input token embedding [batch_size, 1, embed_dim]
        embedded = self.embedding(input_token)

        # Lấy ra hidden state gần nhất
        h_n, c_n = last_hidden
        decoder_hidden = h_n[-1]  # Lấy hidden state của layer cuối cùng [batch_size, hidden_dim]

        # Tính attention context vector
        context_vector, attention_weights = self.attention(encoder_outputs, decoder_hidden, src_mask)

        # Concatenate embedding và context vector
        lstm_input = torch.cat((embedded, context_vector.unsqueeze(1)), dim=2)

        # LSTM forward
        output, new_hidden = self.lstm(lstm_input, last_hidden)

        # Concatenate output với context vector
        output = torch.cat((output.squeeze(1), context_vector), dim=1)  # [batch_size, hidden_dim + encoder_dim]
        output = self.dropout(output)

        # Final projection
        output = self.output_projection(output)  # [batch_size, vocab_size]

        return output, new_hidden, attention_weights

    def forward(self, encoder_outputs, target_tokens, src_mask=None, teacher_forcing_ratio=0.5):
        """
        Forward pass đầy đủ của decoder
        :param encoder_outputs: Outputs từ encoder [batch_size, src_seq_len, encoder_dim]
        :param target_tokens: Target sequence để decode [batch_size, tgt_seq_len]
        :param src_mask: Mask cho source sequence [batch_size, src_seq_len]
        :param teacher_forcing_ratio: Tỷ lệ sử dụng teacher forcing
        """
        batch_size, target_length = target_tokens.size()

        # Khởi tạo hidden state từ encoder outputs
        hidden = self.init_hidden(encoder_outputs)

        # Chuẩn bị tensor để lưu các dự đoán
        outputs = torch.zeros(batch_size, target_length, self.vocab_size).to(target_tokens.device)

        # Token đầu tiên là BOS (Beginning of Sequence)
        input_token = target_tokens[:, 0].unsqueeze(1)  # [batch_size, 1]

        # Lặp qua từng timestep
        for t in range(1, target_length):
            # Forward step cho một token
            output, hidden, _ = self.forward_step(input_token, hidden, encoder_outputs, src_mask)

            # Lưu dự đoán
            outputs[:, t, :] = output

            # Teacher forcing
            use_teacher_forcing = (torch.rand(1).item() < teacher_forcing_ratio)

            if use_teacher_forcing:
                # Sử dụng target token làm input tiếp theo
                input_token = target_tokens[:, t].unsqueeze(1)
            else:
                # Sử dụng dự đoán làm input tiếp theo
                _, top_indices = output.topk(1)
                input_token = top_indices

        return outputs

    def generate(self, encoder_outputs, src_mask=None, max_length=64, bos_id=0, eos_id=2):
        """
        Sinh câu từ encoder outputs (inference)
        """
        batch_size = encoder_outputs.size(0)

        # Khởi tạo hidden state
        hidden = self.init_hidden(encoder_outputs)

        # Bắt đầu với token BOS
        input_token = torch.LongTensor([[bos_id]] * batch_size).to(encoder_outputs.device)

        # Chuẩn bị tensors để lưu tokens và attention weights
        generated_tokens = []
        attention_weights_list = []

        # Sinh tokens cho đến khi đạt max_length hoặc gặp EOS
        for _ in range(max_length):
            # Forward step
            output, hidden, attention_weights = self.forward_step(input_token, hidden, encoder_outputs, src_mask)

            # Lấy token với xác suất cao nhất
            _, top_indices = output.topk(1)

            # Lưu token và attention weights
            generated_tokens.append(top_indices)
            attention_weights_list.append(attention_weights)

            # Input cho bước tiếp theo
            input_token = top_indices

            # Kiểm tra nếu tất cả batch đã sinh EOS token
            if (top_indices == eos_id).all():
                break

        # Stack all tokens and attention weights
        tokens = torch.cat(generated_tokens, dim=1)  # [batch_size, seq_len]
        attention_matrix = torch.stack(attention_weights_list, dim=1)  # [batch_size, seq_len, src_len]

        return tokens, attention_matrix

class CodeBERTLSTM(nn.Module):
    """
    Mô hình encoder-decoder kết hợp CodeBERT và LSTM
    """
    def __init__(self,
                 vocab_size,
                 embed_dim=256,
                 decoder_hidden_dim=256,
                 decoder_layers=2,
                 dropout=0.5,
                 freeze_encoder=False):
        super(CodeBERTLSTM, self).__init__()

        self.encoder = CodeBERTEncoder(freeze_encoder=freeze_encoder)
        self.decoder = LSTMDecoder(
            vocab_size=vocab_size,
            embed_dim=embed_dim,
            encoder_dim=768,  # CodeBERT hidden size
            hidden_dim=decoder_hidden_dim,
            num_layers=decoder_layers,
            dropout=dropout
        )

    def forward(self, input_ids, attention_mask, target_tokens, teacher_forcing_ratio=0.5):
        """
        Forward pass của mô hình encoder-decoder
        """
        # Encoder
        encoder_outputs = self.encoder(input_ids, attention_mask)

        # Decoder
        decoder_outputs = self.decoder(
            encoder_outputs=encoder_outputs,
            target_tokens=target_tokens,
            src_mask=attention_mask,
            teacher_forcing_ratio=teacher_forcing_ratio
        )

        return decoder_outputs

    def generate(self, input_ids, attention_mask, max_length=64, bos_id=0, eos_id=2):
        """
        Sinh câu hỏi từ đoạn code
        """
        # Encoder
        encoder_outputs = self.encoder(input_ids, attention_mask)

        # Decoder generate
        generated_tokens, attention_matrix = self.decoder.generate(
            encoder_outputs=encoder_outputs,
            src_mask=attention_mask,
            max_length=max_length,
            bos_id=bos_id,
            eos_id=eos_id
        )

        return generated_tokens, attention_matrix

# Hàm tính loss
def compute_loss(outputs, targets, pad_id=0):
    """
    Tính cross-entropy loss, bỏ qua các padding tokens
    """
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
    batch_size, seq_len, vocab_size = outputs.size()

    # Reshape outputs và targets
    outputs = outputs.view(-1, vocab_size)
    targets = targets.view(-1)

    loss = criterion(outputs, targets)
    return loss

In [6]:
def train_batch(model, optimizer, batch, tokenizer, device, teacher_forcing_ratio=0.5):
    model.train()
    optimizer.zero_grad()

    # Lấy code và câu hỏi từ batch
    code_batch, question_batch = batch

    # Tokenize đoạn code với CodeBERT tokenizer
    code_inputs = tokenizer(
        code_batch,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to(device)

    # Tokenize câu hỏi với target tokenizer
    question_tokens = torch.tensor(question_batch).to(device)

    # Forward pass
    outputs = model(
        input_ids=code_inputs.input_ids,
        attention_mask=code_inputs.attention_mask,
        target_tokens=question_tokens,
        teacher_forcing_ratio=teacher_forcing_ratio
    )

    # Tính loss
    loss = compute_loss(outputs, question_tokens, pad_id=0)  # Giả sử 0 là pad_id

    # Backpropagation
    loss.backward()

    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    # Cập nhật weights
    optimizer.step()

    return loss.item()

# Sử dụng mô hình cho inference
def generate_question(model, code_snippet, tokenizer, question_tokenizer, device, max_length=64):
    model.eval()

    # Tokenize đoạn code với CodeBERT tokenizer
    code_inputs = tokenizer(
        code_snippet,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to(device)

    # Sinh câu hỏi
    with torch.no_grad():
        generated_ids, attention_matrix = model.generate(
            input_ids=code_inputs.input_ids,
            attention_mask=code_inputs.attention_mask,
            max_length=max_length,
            bos_id=question_tokenizer.bos_token_id,
            eos_id=question_tokenizer.eos_token_id
        )

    # Chuyển đổi ids thành câu
    generated_questions = question_tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )

    return generated_questions, attention_matrix

In [7]:
train_xy_list_small = train_xy_list[:100000]
val_xy_list_small = train_xy_list[51:52]  # Có thể dùng cùng mẫu vì chỉ đang test chạy được không
test_xy_list_small = test_xy_list[:100]  # Chỉ lấy 5 mẫu test

print(f"Small train set size: {len(train_xy_list_small)}")
print(f"Small validation set size: {len(val_xy_list_small)}")
print(f"Small test set size: {len(test_xy_list_small)}")

Small train set size: 100000
Small validation set size: 1
Small test set size: 100


In [ ]:
def check_dictionaries(w2i, i2w):
    """Kiểm tra tính nhất quán giữa các từ điển w2i và i2w"""
    print("\n--- Kiểm tra từ điển ---")
    
    # Kiểm tra các token đặc biệt
    special_tokens = ["<pad>", "<unk>", "<bos>", "<eos>", "<s>", "</s>"]
    for token in special_tokens:
        if token in w2i:
            token_id = w2i[token]
            print(f"{token} -> ID {token_id}", end="")
            
            if token_id in i2w:
                print(f" -> {i2w[token_id]} ✓")
            else:
                print(f" -> MISSING IN i2w ❌")
        else:
            print(f"{token} -> NOT IN w2i ❌")
    
    print(f"\nKích thước w2i: {len(w2i)}")
    print(f"Kích thước i2w: {len(i2w)}")
    print("------------------------\n")

def add_special_tokens_to_dict(w2i, i2w):
    """Bổ sung các token đặc biệt <s> và </s> vào từ điển nếu chúng chưa tồn tại."""
    print("\nBổ sung các token đặc biệt vào từ điển...")
    
    # Thêm token <s>
    if '<s>' not in w2i:
        token_id = len(w2i)
        w2i['<s>'] = token_id
        i2w[token_id] = '<s>'
        print(f"Đã thêm token '<s>' với ID {token_id}")
    else:
        print(f"Token '<s>' đã tồn tại với ID {w2i['<s>']}")
    
    # Thêm token </s>
    if '</s>' not in w2i:
        token_id = len(w2i)
        w2i['</s>'] = token_id
        i2w[token_id] = '</s>'
        print(f"Đã thêm token '</s>' với ID {token_id}")
    else:
        print(f"Token '</s>' đã tồn tại với ID {w2i['</s>']}")
    
    # Kiểm tra lại sau khi thêm
    check_dictionaries(w2i, i2w)
    
    return w2i, i2w

w2i, i2w = add_special_tokens_to_dict(w2i, i2w)



Bổ sung các token đặc biệt vào từ điển...
Đã thêm token '<s>' với ID 60003
Đã thêm token '</s>' với ID 60004

--- Kiểm tra từ điển ---
<pad> -> ID 0 -> <pad> ✓
<unk> -> ID 1 -> <unk> ✓
<bos> -> NOT IN w2i ❌
<eos> -> ID 2 -> <eos> ✓
<s> -> ID 60003 -> <s> ✓
</s> -> ID 60004 -> </s> ✓

Kích thước w2i: 60005
Kích thước i2w: 60005
------------------------



In [9]:
def test_sample_encoding():
    # Lấy một mẫu từ tập huấn luyện
    sample_pair = train_xy_list[0]
    code_text = sample_pair[0][1]  # Lấy văn bản code gốc
    question_text = sample_pair[1][1]  # Lấy câu hỏi gốc
    question_tokens = sample_pair[1][0]  # Lấy tokens của câu hỏi
    
    print("=== KIỂM TRA MÃ HÓA MẪU VÀ ÁNH XẠ TỪ ĐIỂN ===")
    print(f"\n1. Mẫu gốc:")
    print(f"   - Code: {code_text[:100]}...")
    print(f"   - Question: {question_text}")
    
    print("\n2. Tokens của câu hỏi:")
    print(f"   {question_tokens[:20]}...")
    
    # Mã hóa câu hỏi thành chỉ số
    question_indices = [w2i['<bos>']]  # Bắt đầu với BOS token
    token_to_id = {}  # Lưu ánh xạ để kiểm tra
    
    print("\n3. Mã hóa từng token trong câu hỏi:")
    print(f"   {'Token':<15} | {'ID':<8} | {'Lookup lại từ i2w'}")
    print(f"   {'-'*15} | {'-'*8} | {'-'*20}")
    
    print(f"   {'<bos>':<15} | {w2i['<bos>']:<8} | {i2w[w2i['<bos>']]} ({'✓' if i2w[w2i['<bos>']] == '<bos>' else '❌'})")
    
    for token in question_tokens:
        if token == "<eos>":
            idx = w2i["<eos>"]
        elif token in w2i:
            idx = w2i[token]
        else:
            idx = w2i["<unk>"]
        
        question_indices.append(idx)
        token_to_id[token] = idx
        lookup = i2w[idx] if idx in i2w else "MISSING IN i2w"
        check = "✓" if i2w[idx] == token else "❌"
        
        print(f"   {token[:15]:<15} | {idx:<8} | {lookup} ({check})")
        
        if len(question_indices) > 20:  # Giới hạn số lượng token hiển thị
            print("   ... and more ...")
            break
    
    # Kiểm tra giải mã từ indices trở lại thành tokens
    decoded_tokens = []
    for idx in question_indices:
        if idx in i2w:
            token = i2w[idx]
            decoded_tokens.append(token)
    
    print("\n4. Giải mã chỉ số thành tokens:")
    print(f"   {' '.join(decoded_tokens[:20])}...")
    
    # Mã hóa sử dụng Code2QueDataset để so sánh
    print("\n5. Kiểm tra mã hóa từ Code2QueDataset:")
    tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
    dataset = Code2QueDataset([sample_pair], w2i, tokenizer, i2w=i2w)
    sample = dataset[0]
    
    print(f"   - Shape của question_indices: {sample['question_indices'].shape}")
    
    # So sánh một số phần tử đầu tiên của mảng indices
    manually_encoded = question_indices[:10]
    dataset_encoded = sample['question_indices'][:10].tolist()
    
    print("\n6. So sánh kết quả mã hóa thủ công và từ Dataset:")
    print(f"   {'Index':<5} | {'Thủ công':<10} | {'Dataset':<10} | {'Match':<5}")
    print(f"   {'-'*5} | {'-'*10} | {'-'*10} | {'-'*5}")
    
    for i in range(len(manually_encoded)):
        match = "✓" if manually_encoded[i] == dataset_encoded[i] else "❌"
        print(f"   {i:<5} | {manually_encoded[i]:<10} | {dataset_encoded[i]:<10} | {match}")
    
    # Kiểm tra các token đặc biệt
    print("\n7. Kiểm tra các token đặc biệt trong từng từ điển:")
    specials = ["<pad>", "<unk>", "<bos>", "<eos>", "<s>", "</s>"]
    
    print(f"   {'Token':<10} | {'w2i':<10} | {'i2w lookup':<15}")
    print(f"   {'-'*10} | {'-'*10} | {'-'*15}")
    
    for token in specials:
        if token in w2i:
            idx = w2i[token]
            lookup = i2w[idx] if idx in i2w else "MISSING"
            print(f"   {token:<10} | {idx:<10} | {lookup:<15}")
        else:
            print(f"   {token:<10} | {'NOT FOUND':<10} | {'N/A':<15}")
    
    # Kiểm tra tính nhất quán của w2i và i2w
    print("\n8. Kiểm tra tính nhất quán giữa w2i và i2w:")
    consistent = True
    issues = 0
    
    # Kiểm tra xem mọi entry trong w2i có ánh xạ chính xác trong i2w?
    for token, idx in w2i.items():
        if idx not in i2w or i2w[idx] != token:
            print(f"   Không nhất quán: w2i['{token}'] = {idx}, nhưng i2w[{idx}] = {i2w.get(idx, 'MISSING')}")
            consistent = False
            issues += 1
            if issues >= 5:  # Chỉ hiển thị tối đa 5 vấn đề
                print("   ... và nhiều vấn đề khác ...")
                break
    
    if consistent:
        print("   ✓ Không phát hiện vấn đề inconsistency giữa w2i và i2w")
    
    return dataset

## 3. Train mô hình CodeBERT - LSTM

In [ ]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import os
from tqdm import tqdm
import time

from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from rouge import Rouge

class Code2QueDataset(Dataset):
    def __init__(self, xy_list, w2i, tokenizer, i2w=None, max_src_len=128, max_tgt_len=64):
        self.data = xy_list
        self.w2i = w2i
        self.i2w = i2w  # Lưu trữ i2w được truyền vào
        self.tokenizer = tokenizer
        self.max_src_len = max_src_len
        self.max_tgt_len = max_tgt_len
        self.pad_id = w2i['<pad>']
        
        # Thêm BOS token nếu chưa có
        if '<bos>' not in w2i:
            bos_id = len(w2i)
            w2i['<bos>'] = bos_id
            
            # Cập nhật i2w nếu được cung cấp
            if self.i2w is not None:
                self.i2w[bos_id] = '<bos>'
                print(f"Added BOS token with ID {bos_id} to both w2i and i2w")
            
            self.bos_id = bos_id
        else:
            self.bos_id = w2i['<bos>']
            
        # Kiểm tra nếu BOS tồn tại trong w2i nhưng không có trong i2w
        if self.i2w is not None and w2i['<bos>'] not in self.i2w:
            self.i2w[w2i['<bos>']] = '<bos>'
            print(f"Added missing BOS token ID {w2i['<bos>']} to i2w")

        # Kiểm tra và thiết lập EOS token
        if '<eos>' in w2i:
            self.eos_id = w2i['<eos>']
        elif '</s>' in w2i:
            self.eos_id = w2i['</s>']
        else:
            self.eos_id = w2i.get('<eos>', 2)  # Mặc định dùng ID 2 nếu không có

        print(f"Using BOS token ID: {self.bos_id}, EOS token ID: {self.eos_id}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        code_data, question_data = self.data[idx]

        # Lấy văn bản gốc của code và question
        code_text = code_data[1]
        question_tokens = question_data[0]

        # Convert question tokens to indices
        question_indices = [self.bos_id]  # Thêm BOS token
        for token in question_tokens:
            if token == "<eos>":
                question_indices.append(self.eos_id)
            elif token in self.w2i:
                question_indices.append(self.w2i[token])
            else:
                question_indices.append(self.w2i["<unk>"])

        # Padding/truncating question indices
        if len(question_indices) < self.max_tgt_len:
            question_indices += [self.pad_id] * (self.max_tgt_len - len(question_indices))
        else:
            # Đảm bảo token cuối cùng là EOS
            question_indices = question_indices[:self.max_tgt_len-1] + [self.eos_id]

        return {
            'code_text': code_text,
            'question_indices': np.array(question_indices)
        }

def collate_fn(batch):
    """
    Hàm collate cho DataLoader - phiên bản tối ưu
    """
    code_texts = [item['code_text'] for item in batch]

    # Chuyển đổi list của numpy arrays thành một single numpy array trước khi tạo tensor
    question_indices_np = np.stack([item['question_indices'] for item in batch])
    question_indices = torch.tensor(question_indices_np, dtype=torch.long)

    return {
        'code_texts': code_texts,
        'question_indices': question_indices
    }

def train_model(model, train_dataloader, val_dataloader, tokenizer, w2i, i2w,
                device, epochs=10, lr=2e-5, checkpoint_dir='checkpoints',
                patience=3):
    """
    Huấn luyện mô hình Code2Que với tập validation và learning rate scheduler.
    
    Args:
        model: Mô hình CodeBERTLSTM
        train_dataloader: DataLoader cho tập huấn luyện
        val_dataloader: DataLoader cho tập validation
        tokenizer: CodeBERT tokenizer
        w2i: Word-to-index mapping
        i2w: Index-to-word mapping
        device: Thiết bị để huấn luyện (cuda hoặc cpu)
        epochs: Số lượng epochs
        lr: Learning rate ban đầu
        checkpoint_dir: Thư mục lưu checkpoints
        patience: Số epochs chờ khi không có cải thiện validation loss
    """
    # Tạo thư mục checkpoint nếu chưa tồn tại
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
    # Thêm learning rate scheduler
    # ReduceLROnPlateau giảm LR khi validation loss không cải thiện
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min',
        factor=0.5,       # Nhân LR với 0.5 khi cần giảm
        patience=1,       # Số epochs chờ trước khi giảm LR
        verbose=True,     # In ra thông báo khi LR thay đổi
        min_lr=1e-6       # LR tối thiểu
    )
    
    # Early stopping variables
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_path = os.path.join(checkpoint_dir, 'best_model.pt')

    for epoch in range(epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        train_samples = 0

        progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs} [Training]')
        for batch in progress_bar:
            code_texts = batch['code_texts']
            question_indices = batch['question_indices'].to(device)

            code_inputs = tokenizer(
                code_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=128
            ).to(device)

            outputs = model(
                input_ids=code_inputs.input_ids,
                attention_mask=code_inputs.attention_mask,
                target_tokens=question_indices,
                teacher_forcing_ratio=1
            )

            loss = compute_loss(outputs, question_indices, pad_id=w2i["<pad>"])

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            batch_size = question_indices.size(0)
            total_train_loss += loss.item() * batch_size
            train_samples += batch_size

            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'avg_loss': f'{total_train_loss/train_samples:.4f}',
                'lr': f'{optimizer.param_groups[0]["lr"]:.2e}'  # Hiển thị learning rate hiện tại
            })

        train_loss = total_train_loss / train_samples

        # Validation phase
        model.eval()
        total_val_loss = 0
        val_samples = 0

        with torch.no_grad():
            progress_bar = tqdm(val_dataloader, desc=f'Epoch {epoch+1}/{epochs} [Validation]')
            for batch in progress_bar:
                code_texts = batch['code_texts']
                question_indices = batch['question_indices'].to(device)

                code_inputs = tokenizer(
                    code_texts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=128
                ).to(device)

                outputs = model(
                    input_ids=code_inputs.input_ids,
                    attention_mask=code_inputs.attention_mask,
                    target_tokens=question_indices,
                    teacher_forcing_ratio=0.0  # No teacher forcing during validation
                )

                loss = compute_loss(outputs, question_indices, pad_id=w2i["<pad>"])

                batch_size = question_indices.size(0)
                total_val_loss += loss.item() * batch_size
                val_samples += batch_size

                progress_bar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'avg_loss': f'{total_val_loss/val_samples:.4f}'
                })

        val_loss = total_val_loss / val_samples

        # Cập nhật learning rate dựa trên val_loss
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']

        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {train_loss:.4f}')
        print(f'  Validation Loss: {val_loss:.4f}')
        print(f'  Learning Rate: {current_lr:.2e}')

        # Save checkpoint after each epoch
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),  # Lưu trạng thái scheduler
            'train_loss': train_loss,
            'val_loss': val_loss,
            'w2i': w2i,
            'i2w': i2w,
            'lr': current_lr
        }, os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pt'))
        
        # Early stopping and save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # Save best model
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),  # Lưu trạng thái scheduler
                'train_loss': train_loss,
                'val_loss': val_loss,
                'w2i': w2i,
                'i2w': i2w,
                'lr': current_lr
            }, best_model_path)
            print(f"  New best model saved with validation loss: {val_loss:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  Early stopping triggered after {epoch+1} epochs")
                break
                
    return best_model_path

# Đánh giá mô hình bằng các metrics
def evaluate_model(model, test_dataloader, tokenizer, w2i, i2w, device):
    """
    Tính corpus-level BLEU và ROUGE trên toàn bộ tập test.
    """
    model.eval()
    smoothie = SmoothingFunction().method1
    rouge = Rouge()

    all_preds = []  # List[List[str]]
    all_refs  = []  # List[List[List[str]]]  mỗi phần tử là list các tham chiếu cho 1 dự đoán

    # Đảm bảo BOS token cũng có trong i2w
    if '<bos>' in w2i and w2i['<bos>'] not in i2w:
        i2w[w2i['<bos>']] = '<bos>'
        print(f"Added missing BOS token ID {w2i['<bos>']} to i2w dictionary")

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc='Evaluating'):
            code_texts    = batch['code_texts']
            reference_ids = batch['question_indices']  # Tensor [batch, tgt_len]

            # Tokenize code
            code_inputs = tokenizer(
                code_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=test_dataloader.dataset.max_src_len
            ).to(device)

            # Sinh dự đoán
            gen_ids, _ = model.generate(
                input_ids=code_inputs.input_ids,
                attention_mask=code_inputs.attention_mask,
                max_length=test_dataloader.dataset.max_tgt_len,
                bos_id=w2i['<bos>'],
                eos_id=w2i['<eos>']
            )

            # Chuyển IDs → token strings, loại special tokens
            for pred_ids, ref_ids in zip(gen_ids, reference_ids):
                pred_tokens = [
                    i2w[int(idx)] for idx in pred_ids
                    if i2w[int(idx)] not in {'<pad>','<bos>','<eos>','<s>','</s>'}
                ]
                ref_tokens = [
                    i2w[int(idx)] for idx in ref_ids
                    if i2w[int(idx)] not in {'<pad>','<bos>','<eos>','<s>','</s>'}
                ]
                all_preds.append(pred_tokens)
                all_refs.append([ref_tokens])  # mỗi ref nằm trong list để corpus_bleu hiểu

    # Corpus-level BLEU
    corpus_bleu_score = corpus_bleu(
        all_refs,
        all_preds,
        smoothing_function=smoothie
    )

    # ROUGE: vẫn tính bình thường trung bình trên từng cặp
    hyp_texts = [" ".join(p) for p in all_preds]
    ref_texts = [" ".join(r[0]) for r in all_refs]
    rouge_scores = rouge.get_scores(hyp_texts, ref_texts)

    # Trung bình ROUGE
    avg_rouge = {
        metric: {
            m: sum(score[metric][m] for score in rouge_scores) / len(rouge_scores)
            for m in rouge_scores[0][metric]
        }
        for metric in rouge_scores[0]
    }

    print(f"Corpus BLEU: {corpus_bleu_score:.4f}")
    return {
        'bleu': corpus_bleu_score,
        'rouge': avg_rouge
    }

# Set random seed
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Kiểm tra CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Khởi tạo datasets
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
train_dataset = Code2QueDataset(train_xy_list, w2i, tokenizer, i2w=i2w)
val_dataset = Code2QueDataset(val_xy_list, w2i, tokenizer, i2w=i2w)
test_dataset = Code2QueDataset(test_xy_list, w2i, tokenizer, i2w=i2w)
sample = train_dataset[0]
question_indices = sample['question_indices']
print(question_indices)
test_sample_encoding()
# Khởi tạo dataloaders
batch_size = 16  

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

# Khởi tạo model
vocab_size = len(w2i)
model = CodeBERTLSTM(
    vocab_size=vocab_size,
    embed_dim=256,
    decoder_hidden_dim=256,
    decoder_layers=2,
    dropout=0.3,
    freeze_encoder=True  # Freeze encoder cho quá trình đầu tiên để tránh overfitting
).to(device)

# Train model
print("Starting training...")
best_model_path = train_model(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    tokenizer=tokenizer,
    w2i=w2i,
    i2w=i2w,
    device=device,
    epochs=4,
    lr=1e-4,
    checkpoint_dir=MODEL_PATH,
    patience=3  # Early stopping patience
)

# Load best model for evaluation
checkpoint = torch.load(os.path.join(MODEL_PATH, 'checkpoint_epoch_1.pt'))
model.load_state_dict(checkpoint['model_state_dict'])

# Evaluate model
print("Evaluating model...")
metrics = evaluate_model(
    model=model,
    test_dataloader=test_dataloader,
    tokenizer=tokenizer,
    w2i=w2i,
    i2w=i2w,
    device=device
)

## 4. Đánh giá kết quả 
- Dùng corpus-level BLEU, ROUGE để kiểm tra độ tương đồng từ vựng

In [ ]:
print("Evaluation results:")
print(f"BLEU: {(metrics['bleu']):.4f}")
print(f"ROUGE-1 F1: {(metrics['rouge']['rouge-1']['f']):.4f}")
print(f"ROUGE-L F1: {(metrics['rouge']['rouge-l']['f']):.4f}")

Evaluation results:
BLEU: 0.1113
ROUGE-1 F1: 0.3420
ROUGE-L F1: 0.2844
